# Один из возможных алгоритмов скрейпинга

* указать в коде адрес интересующего сайта: откуда вы хотите скачать данные?
для этого используется библиотека requests

* сохранить веб-страницу (html-код страницы)

* выбрать данные, которые нужно собрать (используется BeautifulSoup)

* записать данные в csv-файл.

Если нужно соскрейпить несколько страниц - повторяем процесс для каждой из них.

**Наша задача:** выгрузить недавние новости в датафрейм `pandas`, чтобы потом сохранить все в csv-файл.

Сначала сгрузим весь html-код страницы и сохраним его в отдельную переменную. Для этого нам понадобится библиотека `requests`. Документация: https://requests.readthedocs.io/en/latest/

Импортируем её:

In [1]:
import requests

Сохраним ссылку на главную страницу сайта в переменную `url` для удобства и выгрузим страницу. (Разумеется, это будет работать при подключении к интернету. Если соединение будет отключено, Python выдаст `NewConnectionError`).

In [2]:
url = 'https://nplus1.ru/' # сохраняем
page = requests.get(url) # загружаем страницу по ссылке

Если мы просто посмотрим на объект, мы ничего особенного не увидим:

In [3]:
page  # response 200 - страница загружена

<Response [200]>

**BeautifulSoup** - python-библиотека для синтаксического разбора файлов HTML/XML. В веб-разработке «суп из тегов» (tag soup) - это слово для синтаксически или структурно некорректного HTML, написанного для веб-страницы.

Документация: https://www.crummy.com/software/BeautifulSoup/bs4/doc/

Импортируем функцию `BeautifulSoup` из библиотеки `bs4` (от *beautifulsoup4*) и заберём со страницы `page` код html в виде текста. 

Сохраним в переменную 'soup' весь HTML-код страницы. HTML-код - это "дерево тегов", формирующее контент страницы.

In [ ]:
!pip install bs4

In [4]:
from bs4 import BeautifulSoup 
# название - отсылка к песне про суп из Алисы в стране чудес https://aliceinwonderland.fandom.com/wiki/Turtle_Soup

In [5]:
soup = BeautifulSoup(page.text, 'html')

Если выведем `soup` на экран, мы увидим то же самое, что в режиме разработчика или в режиме происмотра исходного кода (`view-source` через *Ctrl+U* в Google Chrome).

In [6]:
soup

<!DOCTYPE html>
<html lang="ru" prefix="og: http://ogp.me/ns#">
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<title>N + 1 — главное издание о науке, технике и технологиях</title>
<link as="font" crossorigin="" href="https://staticn1.nplus1.ru/fonts/AeonikPro/AeonikPro-Regular.woff2" rel="preload" type="font/woff2"/>
<link as="font" crossorigin="" href="https://staticn1.nplus1.ru/fonts/Spectral/Spectral-Regular.woff" rel="preload" type="font/woff2"/>
<link href="/front-build/css/main.css?id=546f48fb7f6b8393ef89c8a4413ad872" rel="stylesheet"/>
<link href="/front-build/css/app.css?id=8d36d07e1a5ce6409bf55ef6b0510784" rel="stylesheet"/>
<link href="/apple-touch-icon.png" rel="apple-touch-icon" sizes="180x180"/>
<link href="/favicon-32x32.png" rel="icon" sizes="32x32" type="image/png"/>
<link href="/favicon-16x16.png" rel="icon" sizes="16x16" type="image/png"/>
<link href="/site.webmanifest" rel="manifest"/>
<link color="#f26e40" href=

Для просмотра выглядит не очень удобно.  «Причешем» наш `soup` – воспользуемся методом `.prettify()` в сочетании с функцией `print()`.

In [7]:
print(soup.prettify())

<!DOCTYPE html>
<html lang="ru" prefix="og: http://ogp.me/ns#">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <title>
   N + 1 — главное издание о науке, технике и технологиях
  </title>
  <link as="font" crossorigin="" href="https://staticn1.nplus1.ru/fonts/AeonikPro/AeonikPro-Regular.woff2" rel="preload" type="font/woff2"/>
  <link as="font" crossorigin="" href="https://staticn1.nplus1.ru/fonts/Spectral/Spectral-Regular.woff" rel="preload" type="font/woff2"/>
  <link href="/front-build/css/main.css?id=546f48fb7f6b8393ef89c8a4413ad872" rel="stylesheet"/>
  <link href="/front-build/css/app.css?id=8d36d07e1a5ce6409bf55ef6b0510784" rel="stylesheet"/>
  <link href="/apple-touch-icon.png" rel="apple-touch-icon" sizes="180x180"/>
  <link href="/favicon-32x32.png" rel="icon" sizes="32x32" type="image/png"/>
  <link href="/favicon-16x16.png" rel="icon" sizes="16x16" type="image/png"/>
  <link href="/site.webmanifest" rel="manifest"

В такой выдаче ориентироваться гораздо удобнее (но при желании, то же можно увидеть в браузере, на большом экране).

Чтобы сгрузить все новости с главной страницы сайта, нужно собрать все ссылки на страницы с этими новостями. Ссылки в html-файле всегда заключены в тэг `<a></a>` и имеют атрибут `href`. 

**Функция `soup.find('a')`** найдет первый в дереве тег \<a>.
Если нам нужно найти не только первый элемент, а все элементы по определенному признаку, следует использовать функцию **`soup.find_all('a')`**

In [8]:
soup.find_all('a')

[<a class="w-7.5 h-7.5" href="/search">
 <svg class="w-full h-full">
 <use xlink:href="#n1_pupa"></use>
 </svg>
 </a>,
 <a class="col-span-3 lg:col-span-2 col-start-7 lg:col-start-9 flex items-center hover:text-main transition-colors duration-75" href="https://offline.nplus1.ru/">N + offline</a>,
 <a class="col-span-3 lg:col-span-2 col-start-10 lg:col-start-11 flex items-center hover:text-main transition-colors duration-75" href="https://nplus.pro/">N + production</a>,
 <a class="hover:text-main transition-colors duration-75" href="https://nplus1.ru/about">
                 О нас
               </a>,
 <a class="hover:text-main transition-colors duration-75" href="https://nplus1.ru/difficult" target="_blank">
                 Сложность
               </a>,
 <a class="hover:text-main transition-colors duration-75" href="https://nplus1.ru/adv" target="_blank">
                 Рекламодателям
               </a>,
 <a class="hover:text-main transition-colors duration-75" href="https://nplus

In [9]:
len(soup.find_all('a'))

441

In [10]:
for link in soup.find_all('a'):
    print(link.get('href'))

/search
https://offline.nplus1.ru/
https://nplus.pro/
https://nplus1.ru/about
https://nplus1.ru/difficult
https://nplus1.ru/adv
https://nplus1.ru/blog/2022/04/01/samotek
https://nplus1.ru/search?tags=946
https://nplus1.ru/search?tags=869
https://nplus1.ru/search?tags=874
https://nplus1.ru/search?tags=880
https://nplus1.ru/search?tags=768
https://nplus1.ru/search?tags=890
https://nplus1.ru/search?tags=871
https://nplus1.ru/search?tags=876
https://nplus1.ru/search?tags=775
https://nplus1.ru/search?tags=767
https://nplus1.ru/search?tags=771
https://nplus1.ru/search?tags=772
https://nplus1.ru/search?tags=778
https://nplus1.ru/search?tags=917
https://nplus1.ru/search?tags=918
https://nplus1.ru/search?tags=824
https://t.me/nplusone
https://vk.com/nplusone
https://ok.ru/nplus1
https://twitter.com/nplusodin
https://nplus1.ru/about
https://nplus1.ru/difficult
https://nplus1.ru/adv
https://nplus1.ru/blog/2022/04/01/samotek
https://nplus1.ru/search?tags=946
https://nplus1.ru/search?tags=869
https

Ссылок много. Но нам нужны только новости – ссылки, которые начинаются со слова `/news`. Добавим условие: будем выбирать только те ссылки, в которых есть `/news`. Создадим пустой список `urls` и будем добавлять в него только ссылки, которые удовлетворяют этому условию.

In [11]:
urls = []

for link in soup.find_all('a'):
#     print(link)
    if '/news' in link.get('href'):
        if 'https://' in link.get('href'):
            urls.append(link.get('href'))
urls

['https://nplus1.ru/news/2024/06/21/butyrate-producing-bacteria',
 'https://nplus1.ru/news/2024/06/26/reliquary',
 'https://nplus1.ru/news/2024/06/26/any-sugar',
 'https://nplus1.ru/news/2024/06/26/varanus-rosenbergi',
 'https://nplus1.ru/news/2024/06/26/first-panthera-spelaea',
 'https://nplus1.ru/news/2024/06/26/jwst-af-lep-b',
 'https://nplus1.ru/news/2024/06/25/no-patrilocality',
 'https://nplus1.ru/news/2024/06/25/gaia-companions',
 'https://nplus1.ru/news/2024/06/25/c3-isomerisation',
 'https://nplus1.ru/news/2024/06/25/jt-fly',
 'https://nplus1.ru/news/2024/06/25/birds-no-mosquitos',
 'https://nplus1.ru/news/2024/06/25/change-6-home',
 'https://nplus1.ru/news/2024/06/25/extreme-wildfires-xxi',
 'https://nplus1.ru/news/2024/06/25/mindful-gaming',
 'https://nplus1.ru/news/2024/06/25/extended-ketamine',
 'https://nplus1.ru/news/2024/06/25/lagar-velho',
 'https://nplus1.ru/news/2024/06/25/new-viking-ship-in-norway',
 'https://nplus1.ru/news/2024/06/24/shipwreck-bronze-age',
 'https:

In [12]:
len(urls)

72

In [13]:
urls[1]

'https://nplus1.ru/news/2024/06/26/reliquary'

Теперь наша задача сводится к следующему: изучить одну страницу с новостью, научиться из нее вытаскивать текст и всю необходимую информацию, а потом применить весь набор действий к каждой ссылке из `full_urls` в цикле. Посмотрим на новость с индексом 1, у вас может быть другая, новости обновляются.

In [14]:
url0 = urls[1]

page0 = requests.get(url0)
print(page0)

soup0 = BeautifulSoup(page0.text, 'html')
print(soup0)

<Response [200]>
<!DOCTYPE html>
<html lang="ru" prefix="og: http://ogp.me/ns#">
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<title>В Австрии нашли 1500-летний христианский ларец-реликварий. Он сделан из слоновой кости</title>
<link as="font" crossorigin="" href="https://staticn1.nplus1.ru/fonts/AeonikPro/AeonikPro-Regular.woff2" rel="preload" type="font/woff2"/>
<link as="font" crossorigin="" href="https://staticn1.nplus1.ru/fonts/Spectral/Spectral-Regular.woff" rel="preload" type="font/woff2"/>
<link href="/front-build/css/main.css?id=546f48fb7f6b8393ef89c8a4413ad872" rel="stylesheet"/>
<link href="/front-build/css/app.css?id=8d36d07e1a5ce6409bf55ef6b0510784" rel="stylesheet"/>
<link href="/apple-touch-icon.png" rel="apple-touch-icon" sizes="180x180"/>
<link href="/favicon-32x32.png" rel="icon" sizes="32x32" type="image/png"/>
<link href="/favicon-16x16.png" rel="icon" sizes="16x16" type="image/png"/>
<link href="/site.webmanif

В коде каждой страницы с новостью есть часть с мета-информацией: датой, именем автора и проч. Такая информация окружена тэгом `<meta></meta>`. Посмотрим:

In [15]:
soup0.find_all('meta')

[<meta charset="utf-8"/>,
 <meta content="width=device-width, initial-scale=1" name="viewport"/>,
 <meta content="#f26e40" name="msapplication-TileColor"/>,
 <meta content="#ffffff" name="theme-color"/>,
 <meta content="8c90b02c84ac3b72" name="yandex-verification"/>,
 <meta content="b419949322895fc9106e24ed01be58ac" name="pmail-verification"/>,
 <meta content="N + 1 — главное издание о науке, технике и технологиях" name="description"/>,
 <meta content="N + 1 — главное издание о науке, технике и технологиях" property="og:site_name"/>,
 <meta content="В Австрии нашли 1500-летний христианский ларец-реликварий" property="og:title"/>,
 <meta content="https://minio.nplus1.ru/app-images/929495/667bbb0910524_cover_share.jpeg" property="og:image"/>,
 <meta content="https://nplus1.ru/news/2024/06/26/reliquary" property="og:url"/>,
 <meta content="N + 1 — главное издание о науке, технике и технологиях" property="og:description"/>,
 <meta content="article" property="og:type"/>,
 <meta content="202

Из этого списка нам нужны части с именем автора, датой, заголовком и кратким описанием. Воспользуемся поиском по атрибуту `name`. Передадим функции `find_all()` в качестве аргумента словарь с названием и значением атрибута: 

In [19]:
soup0.find_all('meta', {'name' : 'author'}) # например, автор

[<meta content="Михаил Подрезов" name="author"/>]

Теперь выберем единственный элемент полученного списка (с индексом 0):

In [21]:
soup0.find_all('meta', {'name' : 'author'})[0]

<meta content="Михаил Подрезов" name="author"/>

Нам нужно вытащить из этого объекта `content` – имя автора. Посмотрим на атрибуты:

In [22]:
soup0.find_all('meta', {'name' : 'author'})[0].attrs

{'name': 'author', 'content': 'Михаил Подрезов'}

Как получить отсюда `content`? Очень просто, ведь это словарь! А доставать из словаря значение по ключу мы умеем.

In [23]:
author = soup0.find('meta', {'name' : 'author'}).get('content')
author

'Михаил Подрезов'

Аналогичным образом извлечем дату, заголовок и описание.

In [24]:
date = soup0.find_all('meta', {'itemprop' : 'datePublished'})[0].get('content')
title = soup0.find_all('meta', {'property' : 'og:title'})[0].get('content')

In [25]:
title

'В\xa0Австрии нашли 1500-летний христианский ларец-реликварий'

In [28]:
date

'2024-06-26'

Теперь осталось совсем чуть-чуть. Написать готовую функцию для всех проделанных нами действий и применить ее в цикле для всех ссылок в списке `full_urls`. Напишем! Аргументом функции будет ссылка на новость, а возвращать она будет текст новости и всю необходимую информацию (дата, автор, сложность и проч.). Скопируем все строки кода выше.

In [25]:
def GetNews(url0):
    """
    Возвращает кортеж с url0, date, author, title.
    Параметры:
    
    url0 - ссылка на новость (строка).
    """
    page0 = requests.get(url0)
    soup0 = BeautifulSoup(page0.text, 'html')
    
    author = soup0.find_all('meta', {'name' : 'author'})[0].get('content')
    date = soup0.find_all('meta', {'itemprop' : 'datePublished'})[0].get('content')
    title = soup0.find_all('meta', {'property' : 'og:title'})[0].get('content')
     
    return url0, date, author, title

In [26]:
help(GetNews)

Help on function GetNews in module __main__:

GetNews(url0)
    Возвращает кортеж с url0, date, author, title.
    Параметры:
    
    url0 - ссылка на новость (строка).



Уфф. Осталось применить ее в цикле. Но давайте не будем спешить: импортируем функцию `sleep` для задержки, чтобы на каждой итерации цикла, прежде чем перейти к следующей новости, Python ждал несколько секунд. Во-первых, это нужно, чтобы сайт «не понял», чтобы мы его грабим, да еще автоматически. Во-вторых, с небольшой задержкой всегда есть гарантия, что страница прогрузится (сейчас это пока не очень важно, но особенно актуально будет, когда будем обсуждать встраивание в браузер с Selenium). Приступим.

In [27]:
from time import sleep

In [28]:
urls

['https://nplus1.ru/news/2024/04/17/orchid-bees',
 'https://nplus1.ru/news/2024/04/22/lead-mining',
 'https://nplus1.ru/news/2024/04/22/ici-pregnancy',
 'https://nplus1.ru/news/2024/04/22/beta-blockers-after-infarction',
 'https://nplus1.ru/news/2024/04/20/deer-and-prions',
 'https://nplus1.ru/news/2024/04/20/covid-613-days',
 'https://nplus1.ru/news/2024/04/20/watch-yourself',
 'https://nplus1.ru/news/2024/04/20/rotavirus-wins',
 'https://nplus1.ru/news/2024/04/20/use-them',
 'https://nplus1.ru/news/2024/04/19/vasuki-indicus',
 'https://nplus1.ru/news/2024/04/19/limx-dynamics-cl1-update-1',
 'https://nplus1.ru/news/2024/04/19/pregnancy-outcomes-mortality',
 'https://nplus1.ru/news/2024/04/19/para-to-meta-phenols',
 'https://nplus1.ru/news/2024/04/19/mental-health-homelessness',
 'https://nplus1.ru/news/2024/04/19/upwelling-killed-megafauna',
 'https://nplus1.ru/news/2024/04/19/incised-bear-bone',
 'https://nplus1.ru/news/2024/04/19/bombus-impatiens-flooding',
 'https://nplus1.ru/news/

In [30]:
from tqdm import tqdm

In [32]:
news = [] # это будет список из кортежей, в которых будут храниться данные по каждой новости

for link in tqdm(urls[1:]):
    res = GetNews(link)
    news.append(res)
        
    sleep(3) # задержка в 3 секунды

100%|███████████████████████████████████████████| 71/71 [03:59<00:00,  3.38s/it]


In [33]:
news

[('https://nplus1.ru/news/2024/04/22/lead-mining',
  '2024-04-22',
  'Михаил Подрезов',
  'Добытый в\xa0Польше свинец экспортировали в\xa0Древнюю Русь'),
 ('https://nplus1.ru/news/2024/04/22/ici-pregnancy',
  '2024-04-22',
  'Слава Гоменюк',
  'Ингибиторы контрольных точек иммунитета не\xa0повысили риск патологий беременности'),
 ('https://nplus1.ru/news/2024/04/22/beta-blockers-after-infarction',
  '2024-04-22',
  'Слава Гоменюк',
  'Бета-блокаторы не\xa0снизили риск смерти у\xa0пациентов с\xa0инфарктом миокарда'),
 ('https://nplus1.ru/news/2024/04/20/deer-and-prions',
  '2024-04-20',
  'Сергей Коленов',
  'Двух американских охотников назвали первыми возможными жертвами прионной болезни оленей среди людей'),
 ('https://nplus1.ru/news/2024/04/20/covid-613-days',
  '2024-04-20',
  'Слава Гоменюк',
  'Ученые обнаружили новый штамм коронавируса у\xa0пациента с\xa0613-дневным ковидом'),
 ('https://nplus1.ru/news/2024/04/20/watch-yourself',
  '2024-04-20',
  'Олег Лищук',
  'Вероятность алк

Так теперь выглядит первый элемент списка:

Импортируем `pandas` и создадим датафрейм из списка кортежей: 

In [34]:
import pandas as pd

In [38]:
df = pd.DataFrame(news,columns = ['link', 'date', 'author', 'title'])

In [39]:
df.head(10)

,link,date,author,title
0,https://nplus1.ru/news/2024/04/22/lead-mining,2024-04-22,Михаил Подрезов,Добытый в Польше свинец экспортировали в Древн...
1,https://nplus1.ru/news/2024/04/22/ici-pregnancy,2024-04-22,Слава Гоменюк,Ингибиторы контрольных точек иммунитета не пов...
2,https://nplus1.ru/news/2024/04/22/beta-blocker...,2024-04-22,Слава Гоменюк,Бета-блокаторы не снизили риск смерти у пациен...
3,https://nplus1.ru/news/2024/04/20/deer-and-prions,2024-04-20,Сергей Коленов,Двух американских охотников назвали первыми во...
4,https://nplus1.ru/news/2024/04/20/covid-613-days,2024-04-20,Слава Гоменюк,Ученые обнаружили новый штамм коронавируса у п...
5,https://nplus1.ru/news/2024/04/20/watch-yourself,2024-04-20,Олег Лищук,Вероятность алкогольных блэкаутов связали со с...
6,https://nplus1.ru/news/2024/04/20/rotavirus-wins,2024-04-20,Слава Гоменюк,Ротавирус оказался наиболее распространенным в...
7,https://nplus1.ru/news/2024/04/20/use-them,2024-04-20,Олег Лищук,Американские врачи пренебрегли новыми антибиот...
8,https://nplus1.ru/news/2024/04/19/vasuki-indicus,2024-04-19,Сергей Коленов,Индийские палеонтологи описали потенциально кр...
9,https://nplus1.ru/news/2024/04/19/limx-dynamic...,2024-04-19,Андрей Фокин,Робот Limx Dynamics научился по-человечески хо...


Переименуем столбцы в базе.

In [40]:
df.head()

,link,date,author,title
0,https://nplus1.ru/news/2024/04/22/lead-mining,2024-04-22,Михаил Подрезов,Добытый в Польше свинец экспортировали в Древн...
1,https://nplus1.ru/news/2024/04/22/ici-pregnancy,2024-04-22,Слава Гоменюк,Ингибиторы контрольных точек иммунитета не пов...
2,https://nplus1.ru/news/2024/04/22/beta-blocker...,2024-04-22,Слава Гоменюк,Бета-блокаторы не снизили риск смерти у пациен...
3,https://nplus1.ru/news/2024/04/20/deer-and-prions,2024-04-20,Сергей Коленов,Двух американских охотников назвали первыми во...
4,https://nplus1.ru/news/2024/04/20/covid-613-days,2024-04-20,Слава Гоменюк,Ученые обнаружили новый штамм коронавируса у п...


Всё! Сохраняем датафрейм в файл. Для разнообразия сохраним в Excel:

In [36]:
df.to_excel('nplus-news.xlsx', index=False)